In [3]:
import torch
import numpy as np
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel, BertConfig, BertPreTrainedModel, BertForPreTraining, BertForMaskedLM
import torch.nn as nn
from tqdm import tqdm, tqdm_notebook
import os
RUBERT_PATH = '/Users/zhazira/Documents/masters/nlp2020/lab4 bert/rubert_cased_L-12_H-768_A-12_pt'
modelpath = os.path.join(RUBERT_PATH,'pytorch_model.bin')

In [4]:
os.path.isfile(os.path.join(RUBERT_PATH,'pytorch_model.bin'))

True

In [106]:
# tokenizer = BertTokenizer.from_pretrained(os.path.join(RUBERT_PATH,'vocab.txt'))
tokenizer = BertTokenizer.from_pretrained(RUBERT_PATH, do_lower_case=False)
config = BertConfig.from_json_file(os.path.join(RUBERT_PATH,'bert_config.json'))
model = BertForPreTraining.from_pretrained(modelpath, config=config)
# model.eval()
# model = BertForMaskedLM.from_pretrained(modelpath, config=config)
model.eval()

# model = BertForPreTraining.from_pretrained(RUBERT_PATH)


BertForPreTraining(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affin

In [6]:
text = u"Пассиона́рии — в пассиона рной теории этногенеза люди, обладающие врождённой способностью абсорбировать из внешней среды энергии больше, чем это требуется только для личного и видового самосохранения, и выдавать эту энергию в виде целенаправленной работы по видоизменению окружающей их среды. Судят о повышенной пассионарности того или иного человека по характеристике его поведения и психики."
target = u"психики"
tokenized_text = tokenizer.tokenize(text)
print(tokenized_text)
# Mask a token that we will try to predict back with `BertForMaskedLM`
masked_index = tokenized_text.index(target)
tokenized_text[masked_index] = '[MASK]'
print(masked_index)

# Convert token to vocabulary indices
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
# Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
segments_ids = [1] * len(tokenized_text)
# this is for the dummy first sentence. 
segments_ids[0] = 0
segments_ids[1] = 0

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
print(tokens_tensor.shape)
print(tokens_tensor)
segments_tensors = torch.tensor([segments_ids])
# Load pre-trained model (weights)
# model = BertForMaskedLM.from_pretrained(modelpath)
# model.eval()

# Predict all tokens
predictions = model(tokens_tensor, token_type_ids=segments_tensors)
print(predictions[0].shape)
predicted_index = torch.argmax(predictions[0][0][masked_index]).item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])
print("Original:", text)
print("Masked:", " ".join(tokenized_text))
print(predictions[0].shape)
print("Predicted token:", predicted_token)
print("Other options:")
# just curious about what the next few options look like.
for i in range(10):
#     predictions[0,masked_index,predicted_index] = -11100000
    predicted_index = torch.argmax(predictions[0][0][masked_index]).item()
    print(predicted_index)
    predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])
    print(predicted_token)

['Пасс', '##иона', '##́ри', '##и', '—', 'в', 'пасс', '##иона', 'р', '##ной', 'теории', 'этноген', '##еза', 'люди', ',', 'обладающие', 'врожд', '##ённой', 'способностью', 'абсорб', '##ировать', 'из', 'внешней', 'среды', 'энергии', 'больше', ',', 'чем', 'это', 'требуется', 'только', 'для', 'личного', 'и', 'видов', '##ого', 'самос', '##охран', '##ения', ',', 'и', 'выдавать', 'эту', 'энергию', 'в', 'виде', 'целенаправленно', '##й', 'работы', 'по', 'видоизмен', '##ению', 'окружающей', 'их', 'среды', '.', 'Судя', '##т', 'о', 'повышенной', 'пасс', '##иона', '##р', '##ности', 'того', 'или', 'иного', 'человека', 'по', 'характеристике', 'его', 'поведения', 'и', 'психики', '.']
73
torch.Size([1, 75])
tensor([[ 56493,   9042, 114177,    852,    901,    845,  32038,   9042,    873,
           1700,  14422,  99221,  16182,  11894,    128,  62824,  47474,  11827,
          38843,  81836,   7996,   1703,  20895,  18624,  17754,   8980,    128,
           3622,   3998,  23145,   4564,   2748,  23332,  

In [7]:
from torch import load
di = load(modelpath)
# with open(modelpath,encoding='utf-8') as fh:
print(type(di))

<class 'collections.OrderedDict'>


In [9]:
#texts data
texts_df = pd.read_csv('texts_train.txt', sep="\t", header=None)
texts_df.columns = ["text"]
texts_df.head()

,text
0,"Сериал очень люблю, но Академия и Земля вызыва..."
1,"думал, что будет лучше идея очень интересна - ..."
2,с творчеством Головачева я познакомился посред...
3,"то-то я и в большое неудовольствие прочитал ""А..."
4,как мне показалось местами сильно смахивает на...


In [21]:
#texts data
texts_df = pd.read_csv('dataset1000.csv', sep=",")
texts_df.head()

,text,tonality
0,"Сериал очень люблю, но Академия и Земля вызыва...",6
1,"думал, что будет лучше идея очень интересна - ...",7
2,с творчеством Головачева я познакомился посред...,10
3,"то-то я и в большое неудовольствие прочитал ""А...",5
4,как мне показалось местами сильно смахивает на...,6


In [22]:
max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']

def get_means(sentence):
    tokenized_text = tokenizer.tokenize(sentence)
    tokenized_text = tokenized_text[:max_input_length-2]
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    
    segments_ids = [1] * len(tokenized_text)    
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    predictions = model(tokens_tensor, token_type_ids=segments_tensors)
    _, secondDims, thirdDims = predictions[0].shape
    
    finalVector = []
    
    for i in range(secondDims):
        currentArr = predictions[0][0][i].detach().numpy()
        if len(finalVector) == 0:
            finalVector = currentArr
        else:
            finalVector = np.add(finalVector, currentArr)
    return np.mean(finalVector)

In [23]:
get_means(texts_df['text'][0])

-213.32309

In [26]:
texts_df["tonality"]

0       6
1       7
2      10
3       5
4       6
       ..
994     9
995     9
996     6
997     4
998     9
Name: tonality, Length: 999, dtype: int64

In [36]:
os.path.isfile('vectors.csv')

True

In [37]:
scores_df = pd.read_csv('vectors.csv', dtype='float64')
vector = [get_means(sentence) for sentence in texts_df["text"].tolist()]
scores_df['vector'] = vector

scores_df.head()

,tonality,vector
0,NaN,-213.323090
1,NaN,-95.984756
2,NaN,-279.542175
3,NaN,-155.725052
4,NaN,-251.803604


In [41]:
scores_df['tonality'] = texts_df["tonality"]

In [49]:
scores_df.head()

,tonality,vector
0,6,-213.323090
1,7,-95.984756
2,10,-279.542175
3,5,-155.725052
4,6,-251.803604


In [ ]:
scores_df.to_csv('.csv', index = False, header=True)

In [70]:
tonality = np.array(scores_df['tonality'])
vector = np.array(scores_df['vector']).reshape(-1, 1)

In [74]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test  = train_test_split(tonality, vector, test_size = 0.2)

In [75]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
regressor = RandomForestClassifier(n_estimators=2000, random_state=0)
R = regressor.fit(X_train, y_train)
Y_predict = regressor.predict(X_test)

In [80]:
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report

In [83]:
y_test

array([ 9,  9,  6,  8,  2, 10, 10,  9, 10,  7,  9,  7,  9, 10, 10,  7, 10,
        9,  1,  5,  9,  9,  9,  7,  8,  9,  1, 10,  9,  9,  9,  9,  9, 10,
        8,  6,  6, 10,  7, 10,  7, 10,  9,  9, 10,  9, 10,  8,  9,  9,  9,
        4,  7,  9,  9,  9,  8,  8,  9,  9,  9,  9, 10,  8, 10,  3, 10,  9,
       10,  9,  7,  5,  5, 10, 10,  9, 10,  9,  7,  7,  9,  8, 10,  8,  6,
       10,  8, 10,  9, 10,  8,  7, 10,  8,  4,  8,  1, 10,  8,  9,  9, 10,
       10,  9,  8,  8,  9,  9, 10,  9,  9, 10,  2, 10,  9,  8, 10, 10,  9,
       10,  3,  9,  9,  9,  9, 10,  9, 10,  8, 10,  7,  9,  8,  9,  9,  6,
        7, 10,  8, 10, 10,  8,  7,  8,  8, 10,  4, 10, 10,  9,  9, 10,  8,
        3,  8,  9,  9, 10,  9, 10,  6,  8, 10,  9,  9,  8, 10, 10,  8,  4,
        9, 10,  8,  7,  7,  7,  5,  5,  9,  9,  6,  9, 10, 10,  9,  7,  9,
        9,  9, 10,  3,  9, 10,  9, 10, 10, 10,  9, 10, 10])

In [84]:
Y_predict

array([ 8,  1, 10,  8,  9,  9, 10, 10,  4,  8,  4,  5, 10, 10,  1,  3,  8,
       10,  7,  9,  9,  9,  9,  7,  9, 10, 10,  7,  7, 10,  9,  9, 10, 10,
        9,  4,  6, 10, 10,  8,  8,  8,  7,  7, 10, 10, 10,  8,  8,  8,  1,
        6, 10,  9,  7,  8, 10, 10,  9,  9, 10,  7,  3,  8,  9, 10,  6,  9,
        8, 10,  8,  7,  9,  9,  8,  9,  9,  9,  9,  8,  8,  8,  4, 10,  7,
        9, 10,  1, 10,  6, 10, 10,  8, 10,  6,  8,  9, 10,  9, 10,  8,  9,
       10,  3,  9,  9,  9,  9, 10, 10, 10,  7,  7, 10,  9,  7,  9,  9, 10,
        9,  6,  2,  7, 10, 10,  9,  6,  3,  7,  9,  8,  9,  8, 10,  7,  9,
        8, 10,  8,  9,  8,  9,  9,  5,  8,  8, 10,  9,  8, 10, 10,  9,  9,
        8,  9, 10,  7,  8, 10,  4,  9, 10, 10,  4,  9, 10, 10,  7,  8,  9,
        9,  9, 10,  8,  8,  9,  8,  8,  3, 10, 10, 10, 10,  9,  1,  9,  9,
        8,  2, 10,  4,  8, 10,  1, 10,  9,  8, 10, 10,  6])

In [85]:
accuracy_score(y_test, Y_predict) 

0.235

In [87]:
results = confusion_matrix(y_test, Y_predict) 
print('Confusion Matrix :')
print(results)
print('Report : ')
print(classification_report(y_test, Y_predict))

Confusion Matrix :
[[ 0  0  0  0  0  0  1  0  1  1]
 [ 0  0  0  0  0  0  1  0  1  0]
 [ 0  0  0  1  0  1  0  1  0  1]
 [ 0  0  0  0  0  2  0  0  1  1]
 [ 0  0  0  0  0  0  1  2  2  0]
 [ 0  0  0  1  0  1  1  0  2  2]
 [ 0  0  1  0  1  0  1  8  4  3]
 [ 0  0  0  0  1  0  2  9  8  9]
 [ 4  2  2  2  0  1  8  8 18 24]
 [ 2  0  2  3  0  3  3 11 17 18]]
Report : 
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         3
           2       0.00      0.00      0.00         2
           3       0.00      0.00      0.00         4
           4       0.00      0.00      0.00         4
           5       0.00      0.00      0.00         5
           6       0.12      0.14      0.13         7
           7       0.06      0.06      0.06        18
           8       0.23      0.31      0.26        29
           9       0.33      0.26      0.29        69
          10       0.31      0.31      0.31        59

    accuracy                           0.23 

In [102]:
def get_accuracy(Y_predict, test):
    errors = abs(Y_predict - test)
    accuracy = 100 - np.mean(100 * (errors / test))
    print('Accuracy:', round(accuracy, 2), '%.')
    return accuracy

In [103]:
get_accuracy(Y_predict, y_test)

Accuracy: 61.68 %.


61.67876984126984